In [ ]:
import pandas as pd
import splink.comparison_library as cl
from splink import DuckDBAPI, Linker, SettingsCreator, block_on, splink_datasets

In [ ]:
db_api = DuckDBAPI()

In [ ]:
# Read in the data
df = pd.read_csv('clean_data.csv', dtype=str)
df.head()

In [ ]:
df.shape

In [ ]:
# Look at completeness of fields:

from splink.exploratory import completeness_chart
completeness_chart(df, db_api=db_api)

In [ ]:
# Profile distribution of fields:

from splink.exploratory import profile_columns
profile_columns(df, db_api=DuckDBAPI(), top_n=10, bottom_n=5

In [ ]:
# Create the row number column (1 through N)
# We do this because Splink requires a field called 'unique_id' that is a PK for the input dataset (i.e. a field
# that is a unique identifier for each row in the input dataset).

row_numbers = range(1, len(df) + 1)

# Insert as the first column (at index 0)
df.insert(0, 'unique_id', row_numbers)

df.head()

In [ ]:
# This is an example of how to create the settings.
#
# We define 3 things in the settings:
#
#     [1] The 'link_type'. We will always define link_type="dedupe_only" since
#         we will be dealing with a single input file with all the records that need to be linked.
#
#     [2] The 'comparisons'. This specifies which fields from any pair of records will be used when 
#         comparing those records and how that comparison will be done.
#         For example, in the following 'comparisons' we specify that:
#                - The "first_name" field will be compared using the NameComparison function
#                - The "sex" field will be compared looking for an exact match without using term frequency adjustments
#                - The "state" field will be compared looking for an exact match using term frequency adjustments
# 
#     [3] The 'blocking rules'. These define criteria to determine when two records should be compared. It is important to set
#         blocking rules so that we don't compare ALL possible pairs of records, which would result in a number of 
#         comparisons that grows as O(n^2), where n is the total number of records.
     
settings = SettingsCreator(
    link_type="dedupe_only",
    comparisons=[
        cl.NameComparison("first_name"),
        cl.JaroAtThresholds("last_name"),
        cl.ExactMatch("sex").configure(term_frequency_adjustments=False),
        cl.ExactMatch("race").configure(term_frequency_adjustments=False),
        cl.DateOfBirthComparison("birth_date", input_is_string=True),
        cl.DateOfBirthComparison("death_date", input_is_string=True),
        cl.ExactMatch("social_security_number").configure(term_frequency_adjustments=False),
        cl.LevenshteinAtThresholds("address", 1),
        cl.ExactMatch("city").configure(term_frequency_adjustments=True),
        cl.ExactMatch("state").configure(term_frequency_adjustments=True),
        cl.ExactMatch("zip_code").configure(term_frequency_adjustments=True),
        cl.ExactMatch("phone").configure(term_frequency_adjustments=False),
    ],
    blocking_rules_to_generate_predictions=[
        block_on("first_name", "birth_date"),
        block_on("last_name", "birth_date"),
        block_on("social_security_number"),
        block_on("phone"),
        block_on("zip_code", "birth_date"),
        block_on("last_name", "death_date"),
        block_on("address", "birth_date"),
        block_on("first_name", "last_name", "birth_date"),
        block_on("first_name", "last_name", "zip_code")
    ]

In [ ]:
# This creates the linker object that will then be used to train the model:

linker = Linker(df, settings, db_api)

In [ ]:
# Estimate relevant probabilities:

In [ ]:
linker.training.estimate_probability_two_random_records_match(
    [block_on("first_name", "last_name")],
    recall=0.7,
)

In [ ]:
linker.training.estimate_u_using_random_sampling(max_pairs=1e6)

In [ ]:
# Train the model:

training_blocks = [
    block_on("first_name", "last_name"),
    block_on("phone"),
    block_on("address", "birth_date"),
    block_on("last_name", "birth_date"),
    block_on("social_security_number"),
    block_on("zip_code", "birth_date")
]

for rule in training_blocks:
    linker.training.estimate_parameters_using_expectation_maximisation(rule)

In [ ]:
# Make pairwise_predictions using the trained model:

pairwise_predictions = linker.inference.predict(threshold_match_weight=-5)

In [ ]:
# Save the pairwise predictions in a pandas dataframe:

df_pairwise_predictions = pairwise_predictions.as_pandas_dataframe()

In [ ]:
# For example, if we now wanted to see all pairwise predictions with match probability > 0.75 we could do this:

df_pairwise_predictions[df_pairwise_predictions['match_probability'] > 0.75].shape

In [ ]:
# If we want to see how many matches we have with different match probabilities:
df_pairwise_predictions['match_probability'].value_counts().sort_index(ascending=False)

In [ ]:
# Create clusters of records (persons) using the trained model:

clusters = linker.clustering.cluster_pairwise_predictions_at_threshold(
    pairwise_predictions, 0.95
)

In [ ]:
# Save the clusters of people in a pandas dataframe.
# This dataframe shows matches between records because we have assigned a unique cluster_id to
# all records that correspond to the same cluster (same person).

df_clusters = clusters.as_pandas_dataframe() 

In [ ]:
df_clusters.head()

In [ ]:
df_clusters.shape

In [ ]:
df_clusters['cluster_id'].nunique()

In [ ]:
# Save the model to a json file which will be fed to the EMPI application:

linker.misc.save_model_to_json("model.json")